<a href="https://colab.research.google.com/github/sjtalkar/DP-203-Azure-Data-Engineering-Notes/blob/main/SparkAndDeltaLake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [11]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!tar xf 'gdrive/My Drive/Databricks/spark-3.0.0-bin-hadoop3.2.tgz' 
!pip -q install findspark

In [12]:
!pwd

/content


In [10]:
%ls 'gdrive/My Drive/Databricks/spark-3.0.0-bin-hadoop3.2.tgz'

'gdrive/My Drive/Databricks/spark-3.0.0-bin-hadoop3.2.tgz'


In [14]:
%ls /content

gdrive/  sample_data/  spark-3.0.0-bin-hadoop3.2/


In [15]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.12:0.7.0 --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog pyspark-shell'


In [16]:
import findspark
findspark.init()

In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('delta_session').getOrCreate()

In [19]:
history = spark.range(1,1000)
history = history.withColumn("type", lit("history"))
history.show()

+---+-------+
| id|   type|
+---+-------+
|  1|history|
|  2|history|
|  3|history|
|  4|history|
|  5|history|
|  6|history|
|  7|history|
|  8|history|
|  9|history|
| 10|history|
| 11|history|
| 12|history|
| 13|history|
| 14|history|
| 15|history|
| 16|history|
| 17|history|
| 18|history|
| 19|history|
| 20|history|
+---+-------+
only showing top 20 rows



In [20]:
history.write.format("delta").save("delta_sample_data")

In [21]:
!ls -l delta_sample_data

total 12
drwxr-xr-x 2 root root 4096 Nov  7 20:45 _delta_log
-rw-r--r-- 1 root root 3484 Nov  7 20:45 part-00000-db51124c-a038-4501-8671-c34bd59f10ce-c000.snappy.parquet
-rw-r--r-- 1 root root 2746 Nov  7 20:45 part-00001-b432e9cb-a17d-4d0c-9225-17932995ac2b-c000.snappy.parquet
